In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\wazz_\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [6]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# HTML Object
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

NASA Mars News

In [7]:
# Retrieve the first article title and teaser
news_title = soup.find_all('div', class_ = 'content_title')[1].text
news_p = soup.find('div', class_='article_teaser_body').text


print(news_title)
print("------------")
print(news_p)

Another First: Perseverance Captures the Sounds of Driving on Mars
------------
NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.


JPL Mars Space Images - Featured Image

In [8]:
# URL of page to be scraped
image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
main_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
browser.visit(image_url)

# HTML Object 
html_image = browser.html

# Parse HTML with Beautiful Soup
image_soup = bs(html_image, "html.parser")

image_path = image_soup.find('img', class_='headerimage fade-in')['src']


#print(image_path)
featured_image_url = main_url + image_path

print(f'featured_image_url = {featured_image_url}')

featured_image_url = https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


Mars Facts

In [9]:
# URL of page to be scraped
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

# HTML Object
facts_html = browser.html

# Use Pandas to scrape the table
facts_table = pd.read_html(facts_url)

#facts_table

In [10]:
# Create dataframe from the correct table
mars_df = facts_table[0]

# Rename columns
mars_df.columns = ['Description','Mars']

# Set Index to be description
mars_df = mars_df.set_index('Description')

mars_df.head()

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [11]:
# Use Pandas to convert the data to a HTML table string
mars_df.to_html('table.html')

Mars Hemispheres